# Assignment 6  
## Applied Machine Learning

Andrew Chan 
EBE869

This assignment uses the `Suicide Rates Overview 1985 to 2016` dataset from Kaggle: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016. This assumes that you have downloaded the dataset in `master.csv` which is in the same directory as this notebook.

In [63]:
import pandas as pd
import math
import numpy as np

# Locate and load the data file
df = pd.read_csv('master.csv')

# Sanity check
print(f'N rows={len(df)}, M columns={len(df.columns)}')
df.head()

N rows=27820, M columns=12


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [64]:
df.columns.values

array(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype=object)

Adjust the gdp_for_year ($) from string to float.

In [65]:
df[' gdp_for_year ($) ']

0         2,156,624,900
1         2,156,624,900
2         2,156,624,900
3         2,156,624,900
4         2,156,624,900
              ...      
27815    63,067,077,179
27816    63,067,077,179
27817    63,067,077,179
27818    63,067,077,179
27819    63,067,077,179
Name:  gdp_for_year ($) , Length: 27820, dtype: object

In [66]:
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].str.replace(',', '')

In [67]:
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].astype(float) 

In [68]:
df[' gdp_for_year ($) '] 

0        2.156625e+09
1        2.156625e+09
2        2.156625e+09
3        2.156625e+09
4        2.156625e+09
             ...     
27815    6.306708e+10
27816    6.306708e+10
27817    6.306708e+10
27818    6.306708e+10
27819    6.306708e+10
Name:  gdp_for_year ($) , Length: 27820, dtype: float64

# Preprocessing
The assingment states to use the "let's get back to the pre-processed dataset Suicide Rates Overview 1985 to 2016 file", thus we repeat the preprocessing steps here:

## Preprocessing Steps:

### Convert dependent variable to binary values

In [69]:
suicides_mean = df["suicides/100k pop"].mean()
suicides_mean

12.816097411933894

We will create a modify `suicides/100k pop` as follows:  
* All values larger than or equal to `suicides_mean` will be labelled with `high suicide rate` 
* All values smaller than `suicides_mean` will be labelled with `low suicide rate` 

In [70]:
def isHigh(x, mean_val):
    if x > mean_val:
        return 'high suicide rate'
    else:
        return 'low suicide rate'
df['suicides/100k pop'] = df['suicides/100k pop'].apply(isHigh, args=[suicides_mean])
df['suicides/100k pop'].value_counts()

low suicide rate     19061
high suicide rate     8759
Name: suicides/100k pop, dtype: int64

In [71]:
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,low suicide rate,Albania1987,NaN,2.156625e+09,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,low suicide rate,Albania1987,NaN,2.156625e+09,796,Silent
2,Albania,1987,female,15-24 years,14,289700,low suicide rate,Albania1987,NaN,2.156625e+09,796,Generation X
3,Albania,1987,male,75+ years,1,21800,low suicide rate,Albania1987,NaN,2.156625e+09,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,low suicide rate,Albania1987,NaN,2.156625e+09,796,Boomers


### Removal redundant features

#### Remove `suicides_no`

`suicides_no` is highly correlated with with `suicides/100k pop`. Since `suicides/100k pop` was chosen as the dependent variable, it does not make sense to use `suicides_no` since `suicides/100k pop` is $$\frac{suicides\_no}{population/100000}$$ which is a derived attribute. 

Another reason why I chose to keep `suicides/100k pop` is because it helps **normalize suicides by population**. A large country may have many suicides because there are more people that live there, but a smaller country may have more suicides per population. 

In [72]:
df = df.drop(['suicides_no'], axis=1)

#### Remove `gdp_for_year ($)`

`gdp_for_year ($)` is highly correlated with with `gdp_per_capita ($)`. Since `gdp_per_capita ($)` was chosen as the dependent variable, it does not make sense to use `gdp_for_year ($)` since `gdp_per_capita ($)` is $$\frac{gdp\_for\_year ($)}{population}$$ which is a derived attribute.


Another reason why I chose to keep `gdp_per_capita ($)` is because it helps **normalize GDP by population**. A large country may have more gross domestic product because there are more people that live there, but a smaller country may have more gdp per capita. 

In [73]:
df = df.drop([' gdp_for_year ($) '], axis=1)

#### Remove `country-year`

Since `country-year` already is derived from `country` and `year`, it is redundant to include the `country-year`. Thus, we drop `country-year`.

I also chose to keep `country` and `year` because upon one-hot encoding, there will be **N countries** + **M years** columns versus **N x M** additional columns, which should help with computational speed. 

In [74]:
df = df.drop(['country-year'], axis=1)

#### Remove target variable

In [75]:
df_no_target = df.drop(['suicides/100k pop'], axis=1)

In [76]:
df_no_target

,country,year,sex,age,population,HDI for year,gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,312900,NaN,796,Generation X
1,Albania,1987,male,35-54 years,308000,NaN,796,Silent
2,Albania,1987,female,15-24 years,289700,NaN,796,Generation X
3,Albania,1987,male,75+ years,21800,NaN,796,G.I. Generation
4,Albania,1987,male,25-34 years,274300,NaN,796,Boomers
...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,3620833,0.675,2309,Generation X
27816,Uzbekistan,2014,female,75+ years,348465,0.675,2309,Silent
27817,Uzbekistan,2014,male,5-14 years,2762158,0.675,2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,2631600,0.675,2309,Generation Z


### Most Frequent imputation

For all values with NaN, we will replace with most frequent

In [77]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="most_frequent") 
df_no_target[:] = imp.fit_transform(df_no_target)

In [78]:
df_no_target

,country,year,sex,age,population,HDI for year,gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,312900,0.713,796,Generation X
1,Albania,1987,male,35-54 years,308000,0.713,796,Silent
2,Albania,1987,female,15-24 years,289700,0.713,796,Generation X
3,Albania,1987,male,75+ years,21800,0.713,796,G.I. Generation
4,Albania,1987,male,25-34 years,274300,0.713,796,Boomers
...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,3620833,0.675,2309,Generation X
27816,Uzbekistan,2014,female,75+ years,348465,0.675,2309,Silent
27817,Uzbekistan,2014,male,5-14 years,2762158,0.675,2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,2631600,0.675,2309,Generation Z


In [79]:
final_features = df_no_target.columns

In [80]:
final_features

Index(['country', 'year', 'sex', 'age', 'population', 'HDI for year',
       'gdp_per_capita ($)', 'generation'],
      dtype='object')

### Normalization and Standardization

In [81]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_no_target[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']] = scaler.fit_transform(df_no_target[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']])

### One hot encoding of all nominal features

In [82]:
from sklearn.preprocessing import OneHotEncoder
X = df_no_target[['country','year','sex','age','population','HDI for year','gdp_per_capita ($)','generation']].values


In [83]:
ohe = OneHotEncoder()
X = ohe.fit_transform(X[:,[0,2,3,7]].reshape(-1,4)).toarray()

## Final list of features selected

* 'country'
* 'year'
* 'sex'
* 'age'
* 'population'
* 'suicides/100k pop'
* 'HDI for year' 
* 'gdp_per_capita ($)'
* 'generation'

# 1. [20 pts] Due to the severity of this real-world crisis, what information would be the most important one to "machine learn"? Can it be learned?

The most important target variable to machine learn is `suicides/100k pop` as we would like to be able to predict low versus high suicide rates. We would also like to determine what factors may lead to higher suicide rates. Predicting this variable will allow countries to issue multilateral policies to help reduce future deaths. For example, it if it found that `age` is a large factor in suicide rates, countries can focus on preventative social wellness programs for those specific groups. 

# 1. [20 pts] What is the dependent variable you decided? Why?

The dependent variable will be generated from the `suicides/100k pop`. We have chosen to use this versus `suicides_no` because this is normalized to population versus an absolute number of suicides. This will help provide a better prediction so that the model does not skew towards larger countries which may naturally have more suicides. 

# 2. [20 pts] Set the dependent variable into two categories based on a defensible criteria. 
(Hint: skirts of the probability density function)

To attempt a machine learning solution we will turn this into a binary classification problem of `low suicide rate` versus `high suicide rate`. Framing the problem this way will allow us to train a classifier. 

The steps to take will be the following:
1. We will use the dependent variable `suicides/100k pop`
2. Then calculate the mean of this feature `suicides_mean`
3. Convert the data into a binary classification problem such that   
    a. If the value is > than `suicides_mean` then convert to **`high suicide rate`**  
    b. If the value is < than `suicides_mean` then convert to **`low suicide rate`**   

# 3. [20 pts] Develop your classification model(s) to solve your defined problem.

We will create a model using svc and a train test split of 0.80 and 0.20.

Set the x and y values:

In [84]:
df_y = df['suicides/100k pop']
y = df_y.values

In [85]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [86]:
X_train, X_test, y_train, expected = train_test_split(X, y, test_size = 0.2, random_state=0)

---

Try LogisticRegression

In [87]:
from sklearn.linear_model import LogisticRegression
pipe_lr = LogisticRegression(random_state=14,
               penalty='l1',
               solver='liblinear',
               class_weight='balanced',
               multi_class='auto',
               max_iter=10000)
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)

# 4. [20 pts] Evaluate (and report) the model performance(s) using some of the standard techniques (e.g. 80-20 split, 10-fold cross validation, etc.).

In [88]:
print(f'Classification report for classifier {pipe_lr}:\n{metrics.accuracy_score(expected, y_pred)}\n')

Classification report for classifier LogisticRegression(class_weight='balanced', max_iter=10000, penalty='l1',
                   random_state=14, solver='liblinear'):
0.8975557153127247



# 5. [20 pts] Using your classifier model, what is the predicted category of your dependent variable for the input: "year=2000, generation=Generation X, age=20, gender=male"?

In [89]:
input = { 'year':[2000],
         'generation':['Generation X'],
         'age':['15-24 years'],
         'sex':['male'] 
}

df_input = pd.DataFrame(input,columns = ['country', 'year', 'sex', 'age', 'population', 'HDI for year',
       'gdp_per_capita ($)', 'generation'])

In [90]:
df_input

,country,year,sex,age,population,HDI for year,gdp_per_capita ($),generation
0,NaN,2000,male,15-24 years,NaN,NaN,NaN,Generation X


### Imputation

In [91]:
imp.transform(df_input)

array([['Austria', 2000, 'male', '15-24 years', 24000, 0.713, 1299,
        'Generation X']], dtype=object)

In [92]:
df_input[:] = imp.transform(df_input)

In [93]:
df_input

,country,year,sex,age,population,HDI for year,gdp_per_capita ($),generation
0,Austria,2000,male,15-24 years,24000,0.713,1299,Generation X


#### Standardization

In [94]:
df_input[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']] = scaler.transform(df_input[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']])

#### One-hot

In [95]:
X_input = X = df_input[['country','year','sex','age','population','HDI for year','gdp_per_capita ($)','generation']].values

X_input = ohe.transform(X_input[:,[0,2,3,7]].reshape(-1,4)).toarray()

In [96]:
df_input[['country','year','sex','age','population','HDI for year','generation']]

,country,year,sex,age,population,HDI for year,generation
0,Austria,-0.148588,male,15-24 years,-0.465473,-0.324557,Generation X


#### Prediction

In [97]:
pipe_lr.predict(X_input[0].reshape(1, -1))

array(['high suicide rate'], dtype=object)

# 6. [20 pts bonus] Using your (perhaps a different?) model, what is the actual probability of a "Generation X 20-year-old male living in a country with 40000 gdp_per_capita" would commit suicide?

In [98]:
input = {'generation':['Generation X'],
         'gdp_per_capita ($)':[40000],
         'age':['15-24 years'],
         'sex':['male'] 
}

df_input = pd.DataFrame(input,columns = ['country', 'year', 'sex', 'age', 'population', 'HDI for year',
       'gdp_per_capita ($)', 'generation'])

In [99]:
df_input

,country,year,sex,age,population,HDI for year,gdp_per_capita ($),generation
0,NaN,NaN,male,15-24 years,NaN,NaN,40000,Generation X


### Imputation

In [100]:
imp.transform(df_input)

array([['Austria', 2009, 'male', '15-24 years', 24000, 0.713, 40000,
        'Generation X']], dtype=object)

In [101]:
df_input[:] = imp.transform(df_input)

In [102]:
df_input

,country,year,sex,age,population,HDI for year,gdp_per_capita ($),generation
0,Austria,2009,male,15-24 years,24000,0.713,40000,Generation X


#### Standardization

In [103]:
df_input[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']] = scaler.transform(df_input[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']])

#### One-hot

In [104]:
X_input = X = df_input[['country','year','sex','age','population','HDI for year','gdp_per_capita ($)','generation']].values

X_input = ohe.transform(X_input[:,[0,2,3,7]].reshape(-1,4)).toarray()

In [105]:
df_input[['country','year','sex','age','population','HDI for year','generation']]

,country,year,sex,age,population,HDI for year,generation
0,Austria,0.914124,male,15-24 years,-0.465473,-0.324557,Generation X


#### ANSWER: Prediction Probability

In [106]:
pipe_lr.predict_proba(X_input[0].reshape(1, -1))

array([[0.95497437, 0.04502563]])

In [107]:
pipe_lr.predict(X_input[0].reshape(1, -1))

array(['high suicide rate'], dtype=object)